In [25]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
!pip install patool
import patoolib
import scipy.io
import glob
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [60]:
sim_dir =  '/content/gdrive/My Drive/פרויקט מסכם/simulations/'

# for rar in glob.glob(sim_dir + '*.rar'):
#   patoolib.extract_archive(rar, outdir=sim_dir)

/content/gdrive/My Drive/פרויקט מסכם/simulations/two_ellipses.rar
patool: Extracting /content/gdrive/My Drive/פרויקט מסכם/simulations/two_ellipses.rar ...
patool: running /usr/bin/unrar x -- "/content/gdrive/My Drive/פרויקט מסכם/simulations/two_ellipses.rar"
patool:     with cwd='/content/gdrive/My Drive/פרויקט מסכם/simulations/'
patool: ... /content/gdrive/My Drive/פרויקט מסכם/simulations/two_ellipses.rar extracted to `/content/gdrive/My Drive/פרויקט מסכם/simulations/'.
/content/gdrive/My Drive/פרויקט מסכם/simulations/two_spheres.rar
patool: Extracting /content/gdrive/My Drive/פרויקט מסכם/simulations/two_spheres.rar ...
patool: running /usr/bin/unrar x -- "/content/gdrive/My Drive/פרויקט מסכם/simulations/two_spheres.rar"
patool:     with cwd='/content/gdrive/My Drive/פרויקט מסכם/simulations/'
patool: ... /content/gdrive/My Drive/פרויקט מסכם/simulations/two_spheres.rar extracted to `/content/gdrive/My Drive/פרויקט מסכם/simulations/'.
/content/gdrive/My Drive/פרויקט מסכם/simulations/thr

In [48]:
x_train = np.empty([1,9])
for file in glob.glob(homogenous_dir + 'homogenous/*.mat'):
  mat = scipy.io.loadmat(file)['logY'].toarray()
  x_train = np.append(x_train, mat, axis=0)
x_train = x_train[1:]


In [55]:
t_train = np.zeros(x_train.shape[0])
print(t_train.shape)

(2000,)
